# ОПИСАНИЕ ЗАДАЧИ

Данные:
- чековые данные (transactions.parquet, для чтения через pandas дополнительно нужно установить библиотеку pyarrow)
- справочник товаров (materials.csv)
- справочник магазинов (plants.csv)
- справочник клиентов (clients.csv)
Более подробное описание данных дано в файле Data Description.

Цель: 
1) проанализировать данные и определить оптимальную методологию определения отточных клиентов
2) разработать модель вероятности оттока клиентов по выбранной вами методологии
3) дать интерпретацию разработанной модели, ответить на вопросы: какие признаки наиболее влияют на отток клиентов

# ЧАСТЬ 4. FEATURE ENGINEERING

### Загрузка библиотек

In [91]:
import pandas as pd
from pathlib import Path
import numpy as np
import pickle
import gc
from statistics import mode
from tqdm.notebook import tqdm


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")

### Полезные функции

In [39]:
def del_vars(vars_to_del=['tmp', 'tmp2', 'tmp3' 'result']):
    for var in vars_to_del:
        if var in globals():
            del globals()[str(var)]
    gc.collect()

In [40]:
def reduce_to_int(df, cols):   
    for col in cols:
        c_min = df[col].min()
        c_max = df[col].max()
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)  

# 1. Загрузка данных

In [41]:
workdir = str(Path().absolute())

In [42]:
full = pd.read_pickle(workdir+'/data/full.pkl')
data = pd.read_pickle(workdir+'/data/target.pkl')
clients = pd.read_pickle(workdir+'/data/clients.pkl')
materials = pd.read_pickle(workdir+'/data/materials.pkl')

In [43]:
# clnum = 1000
# full = full[full.client_id<clnum]
# data = data[data.client_id<clnum]
# clients = clients[clients.client_id<clnum]

Посмотрим на данные.

In [44]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32094659 entries, 0 to 32094658
Data columns (total 18 columns):
chq_date            datetime64[ns]
chq_position        int32
sales_count         float16
sales_sum           float32
is_promo            bool
client_id           int32
material            int32
plant               int16
chq_id              int32
price               float32
hier_level_1        int8
hier_level_2        int8
hier_level_3        int16
hier_level_4        int16
vendor              int16
is_private_label    bool
is_alco             bool
plant_type          int8
dtypes: bool(3), datetime64[ns](1), float16(1), float32(2), int16(4), int32(4), int8(3)
memory usage: 1.7 GB


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36598170 entries, 0 to 36598169
Data columns (total 7 columns):
chq_date           datetime64[ns]
client_id          int64
chq_count_today    float64
is_active          int32
sum_today          float32
is_high_active     int64
is_churn           float64
dtypes: datetime64[ns](1), float32(1), float64(2), int32(1), int64(2)
memory usage: 1.6 GB


In [46]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99995 entries, 0 to 99994
Data columns (total 4 columns):
gender       99995 non-null int8
birthyear    99995 non-null int32
client_id    99995 non-null int32
city         99995 non-null int8
dtypes: int32(2), int8(2)
memory usage: 1.7 MB


In [47]:
materials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105609 entries, 0 to 105608
Data columns (total 8 columns):
hier_level_1        105609 non-null int8
hier_level_2        105609 non-null int8
hier_level_3        105609 non-null int16
hier_level_4        105609 non-null int16
vendor              105609 non-null int16
is_private_label    105609 non-null bool
is_alco             105609 non-null bool
material            105609 non-null int32
dtypes: bool(2), int16(3), int32(1), int8(2)
memory usage: 2.2 MB


In [48]:
data['is_active'] = data['is_active'].astype(np.int8)
data['is_high_active'] = data['is_high_active'].astype(np.int8)
data['is_churn'] = data['is_churn'].astype(np.int8)

# 2. Feature Engeneering

## _Примечание: часть изначально задуманных признаков были закомментированы, чтобы уменьшить их количество и увеличить скорость дальнейших вычислений._

Группа готовых признаков:
* chq_date - день наблюдений
* client_id	- id клиента
* chq_count_today - количество чеков в день наблюдений
* sum_today	- потраченная сумма в день наблюдений
* is_active	- признак активности в день наблюдений
* is_high_active - признак высокой активности в день наблюдений
* is_churn - целевая переменная оттока 

Группа дополнительных признаков, которые будут созданы:
* номер текущей недели
* агрегации активности клиента по периодам сегодня, на прошлой неделе, за 31 прошлых дней:
    * доля товаров, купленных со скидкой за период
    * (min, max, mean, sum) по количеству чеков за период
    * (min, max, mean, sum) по потраченной сумме за период
    * id магазина с максимальным количеством чеков клиента за период
    * тип магазина с максимальным количеством чеков клиента за период
    * id магазина с максимальной потраченной суммой за период
    * тип магазина с максимальной потраченной суммой за период
    * доля товаров, купленных со скидкой за период
    * доля товаров в чеках за период по группам общих продаж: топовая, высокая, обычная, низкая
    * количество товаров в чеках за период по группам общей частоты продаж: топовая, высокая, обычная, низкая (исключая возвратные транзакции)
    * доля покупок за период самого популярного товара клиента по общей сумме его покупок
    * доля покупок за период самого популярного товара клиента по общему количеству появлений в его чеках (исключая возвратные транзакции)
    * доля купленных товаров за период в типе 1 категории 1
    * доля суммы покупок за период в типе 1 категории 1
    * доля товаров в чеках за период по группам общих продаж в категории 2: топовая, высокая, обычная, низкая
    * доля товаров в чеках за период по группам общей частоты продаж в категории 2: топовая, высокая, обычная, низкая
    * доля покупок клиента за период самого популярного типа в категории 2 по общей сумме покупок
    * доля покупок клиента за период самого популярного типа в категории 2 по общему количеству появлений в его чеках 
    * доля товаров в чеках за период по группам общих продаж в категории 3: топовая, высокая, обычная, низкая
    * доля товаров в чеках за период по группам общей частоты продаж в категории 3: топовая, высокая, обычная, низкая
    * доля покупок клиента за период самого популярного типа в категории 3 по общей сумме его покупок
    * доля покупок клиента за период самого популярного типа в категории 3 по общему количеству появлений в его чеках 
    * доля товаров в чеках за период по группам общих продаж в категории 4: топовая, высокая, обычная, низкая
    * доля товаров в чеках за период по группам общей частоты продаж в категории 4: топовая, высокая, обычная, низкая
    * доля покупок клиента за период самого популярного типа в категории 4 по общей сумме его покупок
    * доля покупок клиента за период самого популярного типа в категории 4 по общему количеству появлений в его чеках 
    * доля товаров в чеках за период по группам общих продаж среди производителей: топовая, высокая, обычная, низкая
    * доля товаров в чеках за период по группам общей частоты продаж среди производителей: топовая, высокая, обычная, низкая
    * доля покупок за период самого популярного производителя клиента по общей сумме его покупок
    * доля покупок за период самого популярного производителя клиента по общему количеству появлений в его чеках 
    * доля возвратных транзакций за период
    * сумма возвратных транзакций за период
    * доля покупок в категории is_private по сумме за период
    * доля покупок в категории is_private по количеству за период
    * доля покупок в категории is_alco по сумме за период
    * доля покупок в категории is_alco по количеству за период
    * доля товаров is_promo за период

### Промежуточный датасет по товарам

Расширим датасет material группировками:

    type1_in_cat1 - принадлежность товара типу 0 в категории 1
    
    is_sales_sum_top - принадлежность к группе top-товары по общей сумме продаж
    is_sales_sum_hgh - принадлежность к группе high-товары по общей сумме продаж
    is_sales_sum_med - принадлежность к группе med-товары по общей сумме продаж
    is_sales_cnt_top - принадлежность к группе top-товары по количеству появлений в чеках
    is_sales_cnt_high - принадлежность к группе high-товары по количеству появлений в чеках
    is_sales_cnt_med - принадлежность к группе med-товары по количеству появлений в чеках
    
    is_sales_sum_cat2_top - принадлежность к группе top-товары в категории 2 по общей сумме продаж
    is_sales_sum_cat2_hgh - принадлежность к группе high-товары в категории 2 по общей сумме продаж
    is_sales_sum_cat2_med - принадлежность к группе med-товары в категории 2 по общей сумме продаж
    is_sales_cnt_cat2_top - принадлежность к группе top-товары в категории 2 по количеству появлений в чеках
    is_sales_cnt_cat2_hgh - принадлежность к группе high-товары в категории 2 по количеству появлений в чеках
    is_sales_cnt_cat2_med - принадлежность к группе med-товары в категории 2 по количеству появлений в чеках
     
    is_sales_sum_cat3_top - принадлежность к группе top-товары в категории 3 по общей сумме продаж
    is_sales_sum_cat3_hgh - принадлежность к группе high-товары в категории 3 по общей сумме продаж
    is_sales_sum_cat3_med - принадлежность к группе med-товары в категории 3 по общей сумме продаж
    is_sales_cnt_cat3_top - принадлежность к группе top-товары в категории 3 по количеству появлений в чеках
    is_sales_cnt_cat3_hgh - принадлежность к группе high-товары в категории 3 по количеству появлений в чеках
    is_sales_cnt_cat3_med - принадлежность к группе med-товары в категории 3 по количеству появлений в чеках
    
    is_sales_sum_cat4_top - принадлежность к группе top-товары в категории 4 по общей сумме продаж
    is_sales_sum_cat4_hgh - принадлежность к группе high-товары в категории 4 по общей сумме продаж
    is_sales_sum_cat4_med - принадлежность к группе med-товары в категории 4 по общей сумме продаж
    is_sales_cnt_cat4_top - принадлежность к группе top-товары в категории 4 по количеству появлений в чеках
    is_sales_cnt_cat4_hgh - принадлежность к группе high-товары в категории 4 по количеству появлений в чеках
    is_sales_cnt_cat4_med - принадлежность к группе med-товары в категории 4 по количеству появлений в чеках
   
    is_sales_sum_vndr_top - принадлежность к группе top-товары среди производителей по общей сумме продаж
    is_sales_sum_vndr_hgh - принадлежность к группе high-товары среди производителей по общей сумме продаж
    is_sales_sum_vndr_med - принадлежность к группе med-товары среди производителей по общей сумме продаж
    is_sales_cnt_vndr_top - принадлежность к группе top-товары среди производителей по количеству появлений в чеках
    is_sales_cnt_vndr_hgh - принадлежность к группе high-товары среди производителей по количеству появлений в чеках
    is_sales_cnt_vndr_med - принадлежность к группе med-товары среди производителей по количеству появлений в чеках

Заранее удалим "лишний" товар 10721

In [49]:
materials = materials[materials.material!=10721]

### type1_in_cat1

In [50]:
materials.rename(columns={'hier_level_1':'is_type1_in_cat1'}, inplace=True)

### is_sales_sum_...

In [51]:
cols = ['material', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'vendor']
suffixes = ['', '_cat2', '_cat3', '_cat4', '_vndr'] 
for suffix, col in zip(suffixes, cols):
    del_vars()
    tmp = full[[col, 'sales_sum']]
    result = tmp.groupby([col])['sales_sum'].sum().reset_index().sort_values(by='sales_sum', ascending=False)
    result['percentile'] = None
    total_sales = result.sales_sum.sum()
    percentile = total_sales/10

    curr_percentile = 0
    i = 1
    for index, row in result.iterrows():
        curr_percentile = curr_percentile + row['sales_sum']
        if curr_percentile >= percentile:
            i = i + 1
            curr_percentile = 0
        result.at[index, 'percentile'] = i
    result[f'is_sales_sum{suffix}_top'] = result['percentile'].apply(lambda x: 1 if x==1 else 0).astype(np.int8)
    result[f'is_sales_sum{suffix}_hgh'] = result['percentile'].apply(lambda x: 1 if x in [2,3,4,5] else 0).astype(np.int8)
#     result[f'is_sales_sum{suffix}_med'] = result['percentile'].apply(lambda x: 1 if x in [6,7,8,9] else 0).astype(np.int8)
    materials = materials.merge(result.drop(columns=['sales_sum','percentile']), how='left', on=col)

### is_sales_cnt_...

In [52]:
cols = ['material', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'vendor']
suffixes = ['', '_cat2', '_cat3', '_cat4', '_vndr'] 
for suffix, col in zip(suffixes, cols):
    del_vars()
    # исключим из выборки возвратные транзакции
    tmp = full[full.sales_sum>=0][[col, 'sales_sum']]

    result = tmp.groupby([col])[col].count().rename('sales_cnt').reset_index().sort_values(by='sales_cnt', ascending=False)
    result['percentile'] = None
    total_sales = result.sales_cnt.sum()
    percentile = total_sales/10

    curr_percentile = 0
    i = 1
    for index, row in result.iterrows():
        curr_percentile = curr_percentile + row['sales_cnt']
        if curr_percentile >= percentile:
            i = i + 1
            curr_percentile = 0
        result.at[index, 'percentile'] = i
    result[f'is_sales_cnt{suffix}_top'] = result['percentile'].apply(lambda x: 1 if x==1 else 0).astype(np.int8)
    result[f'is_sales_cnt{suffix}_hgh'] = result['percentile'].apply(lambda x: 1 if x in [2,3,4,5] else 0).astype(np.int8)
#     result[f'is_sales_cnt{suffix}_med'] = result['percentile'].apply(lambda x: 1 if x in [6,7,8,9] else 0).astype(np.int8)
    materials = materials.merge(result.drop(columns=['sales_cnt','percentile']), how='left', on=col)
    materials = materials.fillna(0)

In [53]:
 reduce_to_int(materials, materials.columns)

### price_mean

In [54]:
result = full.groupby('material')['price'].mean().rename('price_mean').reset_index()
materials = materials.merge(result, how='left', on='material')

### Промежуточный датасет по клиентам

Расширим датасет clients группировками:
    
    best_by_sum_plnt - самый популярный магазин по общей сумме покупок
    best_by_cnt_plnt - самый популярный магазин по количеству покупок

    best_by_sum_plid - самый популярный тип магазина по общей сумме покупок
    best_by_cnt_plid - самый популярный тип магазина по количеству покупок
    
    best_by_sum_mtrl - самый популярный товар клиента по общей сумме его покупок
    best_by_cnt_mtrl - самый популярный товар клиента по общему количеству появлений в его чеках (исключая возвратные транзакции)
    
    best_by_sum_cat2 - самый популярный тип в категории 2 у клиента по общей сумме его покупок
    best_by_cnt_cat2 - самый популярный тип в категории 2 у клиента по общему количеству появлений в его чеках
    
    best_by_sum_cat3 - самый популярный тип в категории 3 у клиента по общей сумме его покупок
    best_by_cnt_cat3 - самый популярный тип в категории 3 у клиента по общему количеству появлений в его чеках
    
    best_by_sum_cat4 - самый популярный тип в категории 4 у клиента по общей сумме его покупок
    best_by_cnt_cat4 - самый популярный тип в категории 4 у клиента по общему количеству появлений в его чеках
    
    best_by_sum_vndr - самый популярный производитель у клиента по общей сумме его покупок
    best_by_cnt_vndr - самый популярный производитель у клиента по общему количеству появлений в его чеках


### best_by_sum_...

In [55]:
cols = ['plant', 'plant_type', 'material', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'vendor']
suffixes = ['_plnt', '_plid', '_mtrl', '_cat2', '_cat3', '_cat4', '_vndr'] 
for suffix, col in zip(suffixes, cols):
    del_vars()
    tmp = full[[col, 'client_id', 'sales_sum']]
    tmp2 = tmp.groupby([col,'client_id'])['sales_sum'].sum()
    result = tmp2.loc[tmp2.groupby(level=1).idxmax()].reset_index()
    clients = clients.merge(result[[col, 'client_id']].rename(columns={col:f'best_by_sum{suffix}'}), 
                            how='left', 
                            on='client_id')

### best_by_cnt_...

In [56]:
cols = ['plant', 'plant_type', 'material', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'vendor']
suffixes = ['_plnt', '_plid', '_mtrl', '_cat2', '_cat3', '_cat4', '_vndr'] 
for suffix, col in zip(suffixes, cols):
    del_vars()
    tmp = full[full.sales_sum>=0][[col, 'client_id', 'sales_sum']]
    tmp2 = tmp.groupby([col,'client_id'])[col].count()
    result = tmp2.loc[tmp2.groupby(level=1).idxmax()].rename('cnt').reset_index()
    clients = clients.merge(result[[col, 'client_id']].rename(columns={col:f'best_by_cnt{suffix}'}), 
                            how='left', 
                            on='client_id')

### Суммирование всех сгенерированных показателей по датам

In [57]:
full['is_private_label'] = full['is_private_label'].astype(np.int8)
full['is_alco'] = full['is_alco'].astype(np.int8)
full['is_promo'] = full['is_promo'].astype(np.int8)

full = full.merge(clients.drop(columns=['gender', 'birthyear', 'city']), how='left', on='client_id')

In [58]:
full = full.merge(materials.drop(columns=['hier_level_2', 
                                          'hier_level_3',
                                          'hier_level_4',
                                          'vendor',
                                          'is_private_label',
                                          'is_alco']), how='left', on='material')

In [59]:
full['is_best_by_sum_plnt'] = (full['best_by_sum_plnt'] == full['plant']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_plid'] = (full['best_by_sum_plid'] == full['plant_type']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_mtrl'] = (full['best_by_sum_mtrl'] == full['material']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_cat2'] = (full['best_by_sum_cat2'] == full['hier_level_2']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_cat3'] = (full['best_by_sum_cat3'] == full['hier_level_3']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_cat4'] = (full['best_by_sum_cat4'] == full['hier_level_4']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_sum_vndr'] = (full['best_by_sum_vndr'] == full['vendor']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_plnt'] = (full['best_by_cnt_plnt'] == full['plant']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_plid'] = (full['best_by_cnt_plid'] == full['plant_type']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_mtrl'] = (full['best_by_cnt_mtrl'] == full['material']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_cat2'] = (full['best_by_cnt_cat2'] == full['hier_level_2']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_cat3'] = (full['best_by_cnt_cat3'] == full['hier_level_3']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_cat4'] = (full['best_by_cnt_cat4'] == full['hier_level_4']).apply(lambda x: 1 if x==True else 0).astype(np.int8)
full['is_best_by_cnt_vndr'] = (full['best_by_cnt_vndr'] == full['vendor']).apply(lambda x: 1 if x==True else 0).astype(np.int8)

In [60]:
full['is_price_discount'] = (full['price_mean'] - full['price']).apply(lambda x: 1 if x>0 else 0).astype(np.int8)

In [61]:
full = full.drop(columns=[
                         'best_by_sum_plnt',
                         'best_by_sum_plid',
                         'best_by_sum_mtrl',
                         'best_by_sum_cat2',
                         'best_by_sum_cat3',
                         'best_by_sum_cat4',
                         'best_by_sum_vndr',
                         'best_by_cnt_plnt',
                         'best_by_cnt_plid',
                         'best_by_cnt_mtrl',
                         'best_by_cnt_cat2',
                         'best_by_cnt_cat3',
                         'best_by_cnt_cat4',
                         'best_by_cnt_vndr',
                         'price',
                         'price_mean',
                         'plant',
                         'plant_type',
                         'hier_level_1',
                         'hier_level_2',
                         'hier_level_3',
                         'hier_level_4',
                         'vendor',
                         'material',
                         'chq_position', 
                         ])

In [62]:
full['transactions_count'] = 1

In [63]:
result = full.groupby(['client_id','chq_date']).agg({
    'transactions_count':'count',
    'is_promo':'sum',
    'is_private_label':'sum',
    'is_alco':'sum',
    'is_type1_in_cat1':'sum',
    'is_sales_sum_top':'sum',
    'is_sales_sum_hgh':'sum',
#     'is_sales_sum_med':'sum',
    'is_sales_sum_cat2_top':'sum',
    'is_sales_sum_cat2_hgh':'sum',
#     'is_sales_sum_cat2_med':'sum',
    'is_sales_sum_cat3_top':'sum',
    'is_sales_sum_cat3_hgh':'sum',
#     'is_sales_sum_cat3_med':'sum',
    'is_sales_sum_cat4_top':'sum',
    'is_sales_sum_cat4_hgh':'sum',
#     'is_sales_sum_cat4_med':'sum',
    'is_sales_sum_vndr_top':'sum',
    'is_sales_sum_vndr_hgh':'sum',
#     'is_sales_sum_vndr_med':'sum',
    'is_sales_cnt_top':'sum',
    'is_sales_cnt_hgh':'sum',
#     'is_sales_cnt_med':'sum',
    'is_sales_cnt_cat2_top':'sum',
    'is_sales_cnt_cat2_hgh':'sum',
#     'is_sales_cnt_cat2_med':'sum',
    'is_sales_cnt_cat3_top':'sum',
    'is_sales_cnt_cat3_hgh':'sum',
#     'is_sales_cnt_cat3_med':'sum',
    'is_sales_cnt_cat4_top':'sum',
    'is_sales_cnt_cat4_hgh':'sum',
#     'is_sales_cnt_cat4_med':'sum',
    'is_sales_cnt_vndr_top':'sum',
    'is_sales_cnt_vndr_hgh':'sum',
#     'is_sales_cnt_vndr_med':'sum',
    'is_best_by_sum_plnt':'sum',
    'is_best_by_sum_plid':'sum',
    'is_best_by_sum_mtrl':'sum',
    'is_best_by_sum_cat2':'sum',
    'is_best_by_sum_cat3':'sum',
    'is_best_by_sum_cat4':'sum',
    'is_best_by_sum_vndr':'sum',
    'is_best_by_cnt_plnt':'sum',
    'is_best_by_cnt_plid':'sum',
    'is_best_by_cnt_mtrl':'sum',
    'is_best_by_cnt_cat2':'sum',
    'is_best_by_cnt_cat3':'sum',
    'is_best_by_cnt_cat4':'sum',
    'is_best_by_cnt_vndr':'sum',
    'is_price_discount':'sum',
}).reset_index()

In [64]:
data = data.merge(result, how='left', on=['client_id', 'chq_date'])

In [65]:
result  = clients[['client_id',
                         'best_by_sum_plnt',
                         'best_by_sum_plid',
                         'best_by_sum_mtrl',
                         'best_by_sum_cat2',
                         'best_by_sum_cat3',
                         'best_by_sum_cat4',
                         'best_by_sum_vndr',
                         'best_by_cnt_plnt',
                         'best_by_cnt_plid',
                         'best_by_cnt_mtrl',
                         'best_by_cnt_cat2',
                         'best_by_cnt_cat3',
                         'best_by_cnt_cat4',
                         'best_by_cnt_vndr',
                 ]]
data = data.merge(result, how='left', on=['client_id'])

Промежуточные датасеты далее не нужны.

In [66]:
del full
del clients
del materials

In [67]:
data = data.fillna(0)

In [68]:
data = data.rename(columns={'chq_count_today':'chq_count', 'sum_today':'chq_sum'})

In [69]:
intcols = ['chq_count', 'transactions_count', 'is_promo',
       'is_private_label', 'is_alco', 'is_type1_in_cat1', 'is_sales_sum_top',
       'is_sales_sum_hgh', 
#            'is_sales_sum_med', 
           'is_sales_sum_cat2_top',
       'is_sales_sum_cat2_hgh', 
#            'is_sales_sum_cat2_med',
       'is_sales_sum_cat3_top', 'is_sales_sum_cat3_hgh',
#        'is_sales_sum_cat3_med',
           'is_sales_sum_cat4_top',
       'is_sales_sum_cat4_hgh', 
#            'is_sales_sum_cat4_med',
       'is_sales_sum_vndr_top', 'is_sales_sum_vndr_hgh',
#        'is_sales_sum_vndr_med', 
           'is_sales_cnt_top', 'is_sales_cnt_hgh',
#        'is_sales_cnt_med', 
           'is_sales_cnt_cat2_top', 'is_sales_cnt_cat2_hgh',
#        'is_sales_cnt_cat2_med', 
           'is_sales_cnt_cat3_top',
       'is_sales_cnt_cat3_hgh',
#            'is_sales_cnt_cat3_med',
       'is_sales_cnt_cat4_top', 'is_sales_cnt_cat4_hgh',
#        'is_sales_cnt_cat4_med',
           'is_sales_cnt_vndr_top',
       'is_sales_cnt_vndr_hgh', 
#            'is_sales_cnt_vndr_med', 
           'is_best_by_sum_plnt',
       'is_best_by_sum_plid', 'is_best_by_sum_mtrl', 'is_best_by_sum_cat2',
       'is_best_by_sum_cat3', 'is_best_by_sum_cat4', 'is_best_by_sum_vndr',
       'is_best_by_cnt_plnt', 'is_best_by_cnt_plid', 'is_best_by_cnt_mtrl',
       'is_best_by_cnt_cat2', 'is_best_by_cnt_cat3', 'is_best_by_cnt_cat4',
       'is_best_by_cnt_vndr', 'is_price_discount']
reduce_to_int(data, intcols)

### номер текущей недели

In [70]:
data['week'] = data['chq_date'].dt.week.astype(np.int8)


###  rate_...

In [71]:
# data[data.client_id==58].head(100)

In [72]:
del_vars()
def get_rolling_amount(grp, freq, col):
    return grp.rolling(freq, on='chq_date')[col].sum()

for col in ['chq_count', 'chq_sum', 'transactions_count']:  
    data[f'{col}_1w'] = data.groupby('client_id', as_index=False, group_keys=False) \
                                .apply(get_rolling_amount, '7D', col)
#     data[f'{col}_2w'] = data[f'{col}_1w'].shift(7)
#     data[f'{col}_3w'] = data[f'{col}_2w'].shift(14)
#     data[f'{col}_4w'] = data[f'{col}_3w'].shift(21)
    data[f'{col}_1m'] = data.groupby('client_id', as_index=False, group_keys=False) \
                                .apply(get_rolling_amount, '31D', col)
    data = data.fillna(0)
    if col in ['chq_count', 'transactions_count']:
        reduce_to_int(data, [f'{col}_1w', 
#                              f'{col}_2w', 
#                              f'{col}_3w', 
#                              f'{col}_4w', 
                             f'{col}_1m' ])

In [73]:
gc.collect()

0

In [74]:
cols = ['is_active', 'is_high_active', 'is_promo',
       'is_private_label', 'is_alco', 'is_type1_in_cat1', 'is_sales_sum_top',
       'is_sales_sum_hgh', 
#         'is_sales_sum_med', 
        'is_sales_sum_cat2_top',
       'is_sales_sum_cat2_hgh', 
#         'is_sales_sum_cat2_med',
       'is_sales_sum_cat3_top', 'is_sales_sum_cat3_hgh',
#        'is_sales_sum_cat3_med', 
        'is_sales_sum_cat4_top',
       'is_sales_sum_cat4_hgh', 
#         'is_sales_sum_cat4_med',
       'is_sales_sum_vndr_top', 'is_sales_sum_vndr_hgh',
#        'is_sales_sum_vndr_med',
        'is_sales_cnt_top', 'is_sales_cnt_hgh',
#        'is_sales_cnt_med', 
        'is_sales_cnt_cat2_top', 'is_sales_cnt_cat2_hgh',
#        'is_sales_cnt_cat2_med', 
        'is_sales_cnt_cat3_top',
       'is_sales_cnt_cat3_hgh',
#         'is_sales_cnt_cat3_med',
       'is_sales_cnt_cat4_top', 'is_sales_cnt_cat4_hgh',
#        'is_sales_cnt_cat4_med', 
        'is_sales_cnt_vndr_top',
       'is_sales_cnt_vndr_hgh', 
#         'is_sales_cnt_vndr_med',
        'is_best_by_sum_plnt',
       'is_best_by_sum_plid', 'is_best_by_sum_mtrl', 'is_best_by_sum_cat2',
       'is_best_by_sum_cat3', 'is_best_by_sum_cat4', 'is_best_by_sum_vndr',
       'is_best_by_cnt_plnt', 'is_best_by_cnt_plid', 'is_best_by_cnt_mtrl',
       'is_best_by_cnt_cat2', 'is_best_by_cnt_cat3', 'is_best_by_cnt_cat4',
       'is_best_by_cnt_vndr', 'is_price_discount']

with tqdm(total=len(cols)) as pbar:
    for col in cols:  
        data[f'{col}_1w'] = data.groupby('client_id', as_index=False, group_keys=False) \
                                    .apply(get_rolling_amount, '7D', col)
#         data[f'{col}_2w'] = data[f'{col}_1w'].shift(7)
#         data[f'{col}_3w'] = data[f'{col}_2w'].shift(14)
#         data[f'{col}_4w'] = data[f'{col}_3w'].shift(21)
        data[f'{col}_1m'] = data.groupby('client_id', as_index=False, group_keys=False) \
                                    .apply(get_rolling_amount, '31D', col)

#         data[f'rate_{col}'] = (data[col] / data['transactions_count']).astype(np.float32)
#         data[f'rate_{col}_1w'] = (data[f'{col}_1w'] / data['transactions_count']).astype(np.float32)
#         data[f'rate_{col}_2w'] = (data[f'{col}_2w'] / data['transactions_count']).astype(np.float32)
#         data[f'rate_{col}_3w'] = (data[f'{col}_3w'] / data['transactions_count']).astype(np.float32)
#         data[f'rate_{col}_4w'] = (data[f'{col}_4w'] / data['transactions_count']).astype(np.float32)
#         data[f'rate_{col}_1m'] = (data[f'{col}_1m'] / data['transactions_count']).astype(np.float32)
        
#         data = data.drop(columns=[col, f'{col}_1w', f'{col}_2w', f'{col}_3w', f'{col}_4w', f'{col}_1m'])
        data = data.fillna(0)
        reduce_to_int(data, [f'{col}_1w', 
#                              f'{col}_2w', 
#                              f'{col}_3w', 
#                              f'{col}_4w', 
                             f'{col}_1m' ])
        gc.collect()
        pbar.update()

In [75]:
data = data.set_index(['client_id', 'chq_date'])

In [76]:
path = str(workdir+'/data')
if not os.path.isdir(path):
    os.mkdir(path)
    print('Папка успешно создана!')
else:
    print('Папка уже существует')
    
data.to_pickle(path+'/data_done.pkl')

Папка уже существует


# 3. Проверка корреляции признаков

Предполагалось выполнить проверку корреляции сформированных признаков. Однако, в связи с нехваткой времени и мощности аппаратных ресурсов, было принято решение пропустить этот этап. Также часть изначально задуманных признаков были закомментированы, чтобы уменьшить их количество и увеличить скорость дальнейших вычислений.

# 4. Сохранение данных

Сохраним финальные данные в формате pickle для последующего использования.

In [ ]:
path = str(workdir+'/data')
if not os.path.isdir(path):
    os.mkdir(path)
    print('Папка успешно создана!')
else:
    print('Папка уже существует')

In [ ]:
data.to_pickle(path+'/data_done.pkl')

In [230]:
# X_train = data.query("(chq_date != '2017-10-04') & (client_id < 5000)").drop(columns='is_churn')
# y_train = data.query("(chq_date != '2017-10-04') & (client_id < 5000)")[['is_churn']]

# X_test = data.query("chq_date == '2017-10-04'").drop(columns='is_churn')
# y_test = data.query("chq_date == '2017-10-04'")[['is_churn']]